# Capstone Project 2-1

## 1.Import package

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

## 2.Website scraping

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
resp = requests.get(url)
soup = BeautifulSoup(resp.text, 'lxml')
table = soup.find(class_="wikitable sortable")
columns = [th.text.replace('\n', '') for th in table.find('tr').find_all('th')]
trs = table.find_all('tr')[1:]
rows = list()
for tr in trs:
    rows.append([td.text.replace('\n', '').replace('\xa0', '') for td in tr.find_all('td')])

## 3. Clean data

### 3-1 Ignore cells with a borough that is Not assigned

In [3]:
df = pd.DataFrame(data=rows, columns=columns)
df = df[df.Borough != 'Not assigned']

### 3-2 More than one neighborhood can exist in one postal code area

In [4]:
group = df.groupby('Postcode')
clean_code = list()
code_cnt = 0 
for postcode in group.groups:
    for cnt in range(group.get_group(postcode).shape[0]):
        if(cnt==0):
            clean_code.append(group.get_group(postcode).values[0])
        else:
            clean_code[code_cnt][2] = clean_code[code_cnt][2] + ',' + group.get_group(postcode).values[cnt][2]
    code_cnt = code_cnt + 1

### 3-3 If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

In [5]:
df.loc[df.Neighbourhood =='Not assigned','Neighbourhood'] = df.loc[df.Neighbourhood =='Not assigned','Borough']

## 4. Convert data to DataFrame

In [6]:
df_clean = pd.DataFrame(clean_code, columns=['Postal Code','Borough','Neighborhood'])
df_clean.shape

(103, 3)